# **Explainability through surrogate models**

Training and comparison of a surrogate tree model versus the original model.

In [50]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.metrics import mean_squared_error
from sklearn.tree import export_text

from tensorflow.keras.models import load_model

DATA_DIR = '../Datasets/'
MODELS_DIR = '../Models/'

# TRAIN_PATH = DATA_DIR + '/Dataset_train.csv'
TRAIN_CF_PATH = DATA_DIR + '/Dataset_train_CF.csv'

# TEST_PATH = DATA_DIR + '/Dataset_test.csv'
TEST_CF_PATH = DATA_DIR + '/Dataset_test_CF.csv'

# MODEL_PATH = MODELS_DIR + 'Pres_hybrid.h5'
MODEL_CF_PATH = MODELS_DIR + 'Pres_hybrid_CF.h5'

SEED = 32

## **Original model**

We start by loading the original model:

In [44]:
model_nn = load_model(MODEL_CF_PATH)

Then, we read the test data y predict the Y values (`y_pred_nn`):

In [45]:
X_train = pd.read_csv(TRAIN_CF_PATH, index_col=0)
X_test = pd.read_csv(TEST_CF_PATH, index_col=0)

X = pd.concat([X_train, X_test])

y_nn = model_nn.predict(X)

print(y_nn.size)
print(y_nn)

18833/18833 [==============================] - 9s 452us/step
602649
[[-0.02186609]
 [-0.27331194]
 [ 0.10668844]
 ...
 [ 0.11868066]
 [ 0.28688934]
 [ 0.20064111]]


## **Tree model**

We train a `DecisionTreeRegressor` based on `model_nn` predictions:

In [46]:
tree = DecisionTreeRegressor(random_state=SEED)

scoring = make_scorer(mean_squared_error, greater_is_better=False)

tree_params = {
    'max_depth': [4, 8, 16, 32, 64, None],
    'criterion': ['squared_error'],
    'splitter': ['best', 'random'],
    'min_samples_split': [2, 8, 16],
    'min_samples_leaf': [1, 2, 8, 16]
}

grid = GridSearchCV(tree, tree_params, scoring=scoring, n_jobs=-1, refit=True, cv=10, verbose=1)

grid.fit(X, y_nn)

model_tree = grid.best_estimator_

print('Best hyperparameters: %s', str(grid.best_params_))

Fitting 10 folds for each of 144 candidates, totalling 1440 fits


Best hyperparameters: %s {'criterion': 'squared_error', 'max_depth': 32, 'min_samples_leaf': 16, 'min_samples_split': 2, 'splitter': 'best'}


# **Results**

Finally, we compare the results between the original model and the surrogate.

In [53]:
# Get tree predictions
y_tree = model_tree.predict(X)

# MSE / RMSE
mse_tree = mean_squared_error(y_nn, y_tree)
rmse_tree = np.sqrt(mse_tree)

# R2
r2_tree = r2_score(y_nn, y_tree)

print(f"MSE tree: {mse_tree}")
print(f"RMSE tree: {rmse_tree}")
print(f"R2 tree: {r2_tree}")


MSE tree: 0.11042773997428136
RMSE tree: 0.3323066956506916
R2 tree: 0.7900170639556732


We also export the resulting tree rules:

In [48]:
tree_rules = export_text(model_tree, feature_names=list(X.columns))
print(tree_rules)

|--- hub_temperature <= -0.05
|   |--- nacelle_angle_c <= -1.23
|   |   |--- V <= 0.81
|   |   |   |--- V <= 0.33
|   |   |   |   |--- hub_temperature <= -0.72
|   |   |   |   |   |--- V <= -0.32
|   |   |   |   |   |   |--- V <= -0.79
|   |   |   |   |   |   |   |--- Out_temperature <= -1.32
|   |   |   |   |   |   |   |   |--- V <= -0.99
|   |   |   |   |   |   |   |   |   |--- W <= -2.17
|   |   |   |   |   |   |   |   |   |   |--- Wind_angle_c <= -0.85
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 5
|   |   |   |   |   |   |   |   |   |   |--- Wind_angle_c >  -0.85
|   |   |   |   |   |   |   |   |   |   |   |--- value: [-0.13]
|   |   |   |   |   |   |   |   |   |--- W >  -2.17
|   |   |   |   |   |   |   |   |   |   |--- Out_temperature <= -1.86
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 7
|   |   |   |   |   |   |   |   |   |   |--- Out_temperature >  -1.86
|   |   |   |   |   |   |   |   |   |   |   |--- truncated bra